In [2]:
import numpy as np
from PIL import Image

In [3]:
#THIS BLOCK IS IMPORTANT


def importClasses():
    classPath = "./tiny-imagenet-200/wnids.txt"
    return open(classPath).read().splitlines()

trainClasses = importClasses()

def createDict(trainClasses):
    #Mapping from class name to class Number
    classDict = {}
    for i in range(len(trainClasses)):
        classDict[trainClasses[i]] = i
    return classDict

classNumDict = createDict(trainClasses)

def getMiddlePatch(path):
    try:
        img = Image.open(path)
    except IOError:
        return
    img = img.convert("RGB")
    img = np.asarray(img)
    return img[3:59, 3:59, :]
    

def pcaAug(original_image):
    #Takes in an image as a np array and returns the pca augmented image as a np array
    renorm_image = np.reshape(original_image,(original_image.shape[0]*original_image.shape[1],3))

    renorm_image = renorm_image.astype('float32')
    renorm_image -= np.mean(renorm_image, axis=0)
    renorm_image /= np.std(renorm_image, axis=0)

    cov = np.cov(renorm_image, rowvar=False)

    lambdas, p = np.linalg.eig(cov)
    alphas = np.random.normal(0, 0.1, 3)

    delta = np.dot(p, alphas*lambdas)

    delta = (delta).astype('int8')

    pca_color_image = np.maximum(np.minimum(original_image + delta, 255), 0).astype('uint8')
    return pca_color_image


def transToOneHot(numClass, dims = 200):
    vec = np.zeros(dims)
    vec[numClass] = 1
    return vec

def generateImgs(path, outClassNum):
    #Takes in path of Image and generates 5 patches and their mirror images and returns them as a list of np arrays and an array of one hot encoded output
    try:
        img = Image.open(path)
    except IOError:
        return
    img = img.convert("RGB")
    img = np.asarray(img)
    arr = [img[0:56, 0:56, :], img[8:, 0:56, :], img[0:56, 8:, :], img[8:, 8:, :], img[3:59, 3:59, :]]
    arr = arr + [np.flip(arr[0], 1),np.flip(arr[1], 1), np.flip(arr[2], 1), np.flip(arr[3], 1), np.flip(arr[4], 1)]
    pcaArr = []
    for i in range(len(arr)):
        pcaArr.append(pcaAug(arr[i]))
    inputArr = arr + pcaArr
    oneHot = transToOneHot(outClassNum, dims = 200)
    outArr = []
    for i in range(len(inputArr)):
        outArr.append(oneHot)
    return inputArr, outArr

def getTrainingSample():
    #Returns one batch of training samples using any 5 random images. Returns a batch of 100
    inputArray = []
    outputArray = []
    for i in range(5):
        outClassNum = np.random.randint(len(trainClasses))
        path = "./tiny-imagenet-200/train/" + str(trainClasses[outClassNum]) + "/images/" + str(trainClasses[outClassNum]) + "_" + str(np.random.randint(100)) + ".JPEG"
        inArr, outArr = generateImgs(path, outClassNum)
        inputArray.append(inArr)
        outputArray.append(outArr)
    temp = np.array(100,3)
    temp[:, 0] = inputArray
    temp[:,1] = outputArray
    temp[:,2] = range(100)
    np.random.shuffle(temp)
    inputArray = temp[:,0]
    outputArray = temp[:,1]
    return inputArray, outputArray

def getOutputArray():
    outputArray = []
    outputTextPath = "./tiny-imagenet-200/val/val_annotations.txt"
    textFile = open(outputTextPath).read().split()
    for i in range(1000):
        outputArray.append(transToOneHot(classNumDict[textFile[6*i+1]], dims = 200))
    return outputArray

def getTestSample():
    inputArray = []
    outputArray = getOutputArray()
    
    
    for i in range(1000):
        path = "./tiny-imagenet-200/val/images/val_" + str(i) + ".JPEG"
        img = getMiddlePatch(path)
        inputArray.append(img)
        
    return inputArray, outputArray